#### This notebook is to use Qiime2-2020.2 environment to construct phylogenetic tree, assign taxonomy, rarefaction, calculate alpha and beta diversities and generate PCoA plots

In [ ]:
### 1. import files
qiime tools import \
--input-path ../data/57316_mros_deblur_otus_unrare.biom \
--type 'FeatureTable[Frequency]' \
--input-format BIOMV210Format \
--output-path ../data/57316_mros_deblur_otus_unrare.qza

qiime tools import \
--input-path ../Qiita_Study11274_ID57316/57316_reference-hit.seqs.fa \
--output-path ../data/57316_reference-hit.seqs.qza \
--type "FeatureData[Sequence]" 

qiime tools import \
  --input-path ../Qiita_Study11274_ID57316/57316_insertion_tree.relabelled.tre \
  --output-path ../data/57316_sepp_tree.qza \
  --type 'Phylogeny[Rooted]'
    
# filter feature table that only contains fragments that are in the insertion tree    
qiime fragment-insertion filter-features \
    --i-table ../data/57316_mros_deblur_otus_unrare.qza \
    --i-tree ../data/57316_sepp_tree.qza \
    --o-filtered-table ../data/57316_filtered-table-deblur.qza \
    --o-removed-table ../data/57316_removed-table.qza    

In [ ]:
### 2. assign taxonomy
wget https://github.com/BenKaehler/readytowear/raw/master/data/gg_13_8/515f-806r/human-stool.qza
wget https://github.com/BenKaehler/readytowear/raw/master/data/gg_13_8/515f-806r/ref-seqs-v4.qza
wget https://github.com/BenKaehler/readytowear/raw/master/data/gg_13_8/515f-806r/ref-tax.qza

qiime feature-classifier fit-classifier-naive-bayes \
--i-reference-reads ../data/ref-seqs-v4.qza \
--i-reference-taxonomy ../data/ref-tax.qza \
--i-class-weight ../data/human-stool.qza \
--o-classifier ../data/gg138_v4_human-stool_classifier.qza

qiime feature-classifier classify-sklearn \
  --i-reads ../data/57316_reference-hit.seqs.qza \
  --i-classifier ../data/gg138_v4_human-stool_classifier.qza \
  --o-classification ../data/57316_bespoke-taxonomy.qza
    
qiime metadata tabulate \
  --m-input-file ../data/57316_bespoke-taxonomy.qza \
  --m-input-file ../data/57316_reference-hit.seqs.qza \
  --o-visualization ../visu/57316_bespoke-taxonomy.qzv
    
qiime tools export \
  --input-path ../data/57316_bespoke-taxonomy.qza \
  --output-path ../data/57316_deblur_taxonomy    

In [ ]:
### 3. Alpha rarefaction 
qiime diversity alpha-rarefaction \
    --i-table ../data/57316_filtered-table-deblur.qza \
    --i-phylogeny ../data/57316_sepp_tree.qza \
    --p-max-depth 50000 \
    --m-metadata-file ../data/mapping_MrOS_add.txt \
    --o-visualization ../visu/57316-alpha-rarefaction.qzv  

In [ ]:
### 4. Compute alpha and beta diversities (lose one sample at 11111)
qiime diversity core-metrics-phylogenetic \
    --i-table ../data/57316_filtered-table-deblur.qza \
    --i-phylogeny ../data/57316_sepp_tree.qza \
    --p-sampling-depth 11111 \
    --m-metadata-file ../data/mapping_MrOS_add.txt \
    --p-n-jobs 1 \
    --output-dir ../data/57316-core-metrics-results
    
# export alpha and beta diversities
qiime tools export \
  --input-path ../data/57316-core-metrics-results/faith_pd_vector.qza \
  --output-path ../data/57316-alpha_PD

qiime tools export \
  --input-path ../data/57316-core-metrics-results/unweighted_unifrac_pcoa_results.qza \
  --output-path ../data/57316-unweighted_unifrac_pcoa_results

qiime tools export \
  --input-path ../data/57316-core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --output-path ../data/57316-unweighted_unifrac_distance

In [ ]:
# rarefy feature table
qiime feature-table rarefy \
--i-table ../data/57316_filtered-table-deblur.qza \
--p-sampling-depth 11111 \
--o-rarefied-table ../data/57316_mros_deblur_otus_rare.qza

# convert qza to biom
qiime tools export \
--input-path ../data/57316_mros_deblur_otus_rare.qza \
--output-path ../data/57316_mros_otus_rare_exp

# convert biom to txt
biom convert -i ../data/57316_mros_otus_rare_exp/feature-table.biom \
  -o ../data/57316_mros_otus_rare_exp/57316_feature-table-rare.txt \
  --to-tsv